# Lab 07

## Zadanie 1

Przygotuj demo programu znajdującego miejsca zerowe metodą Newtona. Wykorzystując `argparse` ([link](https://docs.python.org/3.6/library/argparse.html#module-argparse)) lub `optparse` ([link](https://docs.python.org/3.6/library/optparse.html)) obsłuż:
- ustalane punktu startowego,
- wielkość kroku w pochodnej,
- ilość kroków metody,
- dokładność
- pomoc

Program uruchamiamy podając, np.:

`./newton.py x**2+x+1 -h 0.00001`

In [3]:
%%writefile newton.py
import argparse
import sympy as sp

parser = argparse.ArgumentParser(description="Metoda Newtona do znajdowania miejsc zerowych funkcji.")
parser.add_argument("funkcja", type=str, help="Funkcja w postaci wyrażenia, np. x**2+x+1")
parser.add_argument("start", type=float, help="Punkt startowy dla metody Newtona")
parser.add_argument("-s", "--step", type=float, default=1e-5, help="Wielkość kroku w pochodnej")
parser.add_argument("-n", "--steps", type=int, default=100, help="Maksymalna liczba kroków")
parser.add_argument("-t", "--tolerance", type=float, default=1e-7, help="Tolerancja dla wyniku")

args = parser.parse_args()

x = sp.symbols('x')
f = sp.sympify(args.funkcja)
df = sp.diff(f, x) 

current_x = args.start
for step in range(args.steps):
    f_val = f.evalf(subs={x: current_x})
    df_val = df.evalf(subs={x: current_x})

    if abs(f_val) < args.tolerance:
        print(f"Znaleziono miejsce zerowe: x = {current_x:.10f}, w {step + 1} krokach.")
        break

    if df_val == 0:
        print("Pochodna jest zerowa. Metoda Newtona nie działa.")
        break

    current_x = current_x - f_val / df_val
else:
    print("Nie znaleziono miejsca zerowego w podanej liczbie kroków.")



Overwriting newton.py


In [4]:
!python newton.py "x**2 - 4" 3 -t 1e-8
!python newton.py "cos(x) - x" 1 -t 1e-9
!python newton.py "x**4 - 4*x**2" 1 -t 1e-7

Znaleziono miejsce zerowe: x = 2.0000000000, w 5 krokach.
Znaleziono miejsce zerowe: x = 0.7390851334, w 4 krokach.
Znaleziono miejsce zerowe: x = 0.0001194941, w 13 krokach.


## Zadanie 2

Zadanie należy wykonać wykorzystując program `BagOfWords` z poprzednich zajęć.


Przerób ją tak, żeby interpunkcja, cyfry i wszelkie inne znaki nie przeszkadzały w parsowaniu tekstu. Uruchom ja na [tekście hamleta](http://www.gutenberg.org/cache/epub/1787/pg1787.txt). Ile razy występuje słowo hamlet? Jak brzmi 10 najczęściej występujących słów?

In [5]:
import _io
import re
from collections import Counter
class BagOfWords(object):
    def __init__(self, data):
        self._counter = Counter()
        self._iter_index = 0
        if type(data) == str:
            for s in data.split(" "):
                self._counter[s] += 1
        elif isinstance(data, _io.TextIOWrapper):
            for line in data:
                for w in re.sub(r'[^a-zA-Z\s]', ' ', line.strip()).split(" "):
                    w = w.strip()
                    if w:
                        self._counter[w] += 1
        else:
            raise ValueError("Daj plik albo string")
    def __str__(self):
        output = ""
        for k, v in self._counter.items():
            output += f'{k}:{v}, '
        return output[:-2]
    def __contains__(self, item):
        return item in self._counter.keys()
    
    def __iter__(self):
        self._iter_index = 0
        return self
    def __next__(self):
        keys = list(self._counter.most_common())
        if self._iter_index < len(keys):
            word = keys[self._iter_index][0] 
            self._iter_index += 1
            return word
        else:
            raise StopIteration
        
    def __add__(self, other):
        if not isinstance(other, BagOfWords):
            raise ValueError("Złe dodawanie")
        res = self._counter + other._counter
        new_bag = BagOfWords("")
        new_bag._counter = res 
        return new_bag
    
    def __getitem__(self, key):
        if type(key) != str:
            raise ValueError("Zły typ klucza")
        return self._counter[key]
    def __setitem__(self, key, value):
        if type(key) != str or type(value) != int:
            raise ValueError("Zły typ klucza lub wartosci")
        self._counter[key] = value

In [6]:
slownik = BagOfWords(open("hamlet.txt"))
print(slownik["Hamlet"])

for i, w in enumerate(slownik):
    if i == 10:
        break
    else:
        print(w, ": ", slownik[w])


119
the :  1141
and :  789
of :  782
to :  739
I :  608
you :  563
a :  547
in :  479
my :  444
it :  381


## Zadanie 3

Wykorzystując `pickle` zapisz i odczytaj klasy z poprzedniego zadania nakarmonej Hamletem. Porównaj metody i rozmiar.

In [7]:
import pickle
with open("hamlet.pkl", "wb") as file:
    pickle.dump(slownik, file)

In [8]:
with open("hamlet.pkl", "rb") as file:
    nowy_slownik = pickle.load(file)

## Zadanie 4

Wykorzystując [https://gist.github.com/pamelafox/986163](https://gist.github.com/pamelafox/986163) podaj aktualną godzinę we:
- wszystkich krajach, wyświetlając je zgrupowane względem kontynentów,
- przeprowadź symulacyjne wyświetlanie kolejnych krajów w miarę jak w danym kraju wybija północ, opóżnienie wyświetlania ustaw proporcjonalne do realnego czasu

## Zadanie 5

Dla klasy `BagOfWords` napisz metody `save` oraz `load` wykorzystujące `json`'a do zapisu i odczytu danych.

In [9]:
import _io
import re
import json
from collections import Counter
class BagOfWords(object):
    def __init__(self, data):
        self._counter = Counter()
        self._iter_index = 0
        if type(data) == str:
            for s in data.split(" "):
                self._counter[s] += 1
        elif isinstance(data, _io.TextIOWrapper):
            for line in data:
                for w in re.sub(r'[^a-zA-Z\s]', ' ', line.strip()).split(" "):
                    w = w.strip()
                    if w:
                        self._counter[w] += 1
        else:
            raise ValueError("Daj plik albo string")
    def __str__(self):
        output = ""
        for k, v in self._counter.items():
            output += f'{k}:{v}, '
        return output[:-2]
    def __contains__(self, item):
        return item in self._counter.keys()
    
    def __iter__(self):
        self._iter_index = 0
        return self
    def __next__(self):
        keys = list(self._counter.most_common())
        if self._iter_index < len(keys):
            word = keys[self._iter_index][0] 
            self._iter_index += 1
            return word
        else:
            raise StopIteration
        
    def __add__(self, other):
        if not isinstance(other, BagOfWords):
            raise ValueError("Złe dodawanie")
        res = self._counter + other._counter
        new_bag = BagOfWords("") 
        new_bag._counter = res 
        return new_bag
    
    def __getitem__(self, key):
        if type(key) != str:
            raise ValueError("Zły typ klucza")
        return self._counter[key]
    def __setitem__(self, key, value):
        if type(key) != str or type(value) != int:
            raise ValueError("Zły typ klucza lub wartosci")
        self._counter[key] = value

    def save(self, path):
        with open(path, 'w') as file:
            json.dump(self._counter, file)
    
    def load(self, path):
        with open(path, 'r') as file:
            dict = json.load(file)
            self._counter = Counter()
            for k, v in dict.items():
                self._counter[k] = v

In [10]:
slownik = BagOfWords(open("hamlet.txt"))
slownik.save("dzejson.json")
slownik.load("dzejson.json")
for k in slownik:
    print(k, ": ", slownik[k])

the :  1141
and :  789
of :  782
to :  739
I :  608
you :  563
a :  547
in :  479
my :  444
it :  381
is :  374
Ham :  358
not :  329
d :  306
this :  296
that :  291
his :  286
with :  284
And :  274
s :  247
me :  235
your :  234
be :  233
for :  213
lord :  205
him :  197
The :  185
have :  176
King :  174
or :  172
as :  169
he :  169
will :  162
do :  151
so :  149
are :  147
but :  147
on :  142
by :  139
That :  136
To :  135
But :  131
all :  127
we :  121
O :  120
Hamlet :  119
our :  116
shall :  112
What :  110
what :  110
Queen :  109
Hor :  109
no :  107
from :  106
Project :  102
Gutenberg :  99
A :  99
at :  95
t :  95
thou :  93
more :  92
You :  87
may :  86
Pol :  86
her :  83
was :  81
thy :  81
good :  80
like :  77
know :  77
If :  76
For :  76
ll :  76
they :  76
My :  76
most :  75
them :  75
As :  74
us :  73
now :  72
come :  71
Enter :  70
father :  69
if :  67
must :  67
would :  67
love :  67
This :  66
It :  64
well :  64
sir :  63
there :  62
th :  62
Laer